# 循环神经网络的从零开始实现

## 独热编码

* 选项一：每个词元都表示为一个数字索引
* 选项二：更具表现力（more expressive）的特征向量
* （批量大小，时间步数）
* （时间步数，批量大小，词表大小）

## 循环神经网络模型

`init_rnn_state`函数

```
def init_rnn_state(batch_size, num_hiddens, device):
    return (torch.zeros((batch_size, num_hiddens), 
            device=device), )
```

为什么使用元组

```
# `inputs` shape: 
# (`num_steps`, `batch_size`, `vocab_size`)
# Shape of `X`: (`batch_size`, `vocab_size`)
for X in inputs:
    H = torch.tanh(torch.mm(X, W_xh) + torch.mm(
                    H, W_hh) + b_h)
    Y = torch.mm(H, W_hq) + b_q
    outputs.append(Y)
```

## 梯度裁剪

* 对于长度为$T$的序列，在迭代中计算这$T$个时间步上的梯度，将会在反向传播过程中产生长度为$\mathcal{O}(T)$的矩阵乘法链
* 当$T$较大时，它可能导致数值不稳定，例如可能导致梯度爆炸或梯度消失



* *利普希茨连续的*（Lipschitz continuous）

$$|f(\mathbf{x}) - f(\mathbf{y})| \leq L \|\mathbf{x} - \mathbf{y}\|$$

* 通过$\eta \mathbf{g}$更新参数向量

$$|f(\mathbf{x}) - f(\mathbf{x} - \eta\mathbf{g})| \leq L \eta\|\mathbf{g}\|$$


* 将梯度$\mathbf{g}$投影回给定半径（例如$\theta$）的球

$$\mathbf{g} \leftarrow \min\left(1, \frac{\theta}{\|\mathbf{g}\|}\right) \mathbf{g}$$

* 梯度范数永远不会超过$\theta$


* the updated gradient is entirely aligned with the original direction of $\mathbf{g}$
* Gradient clipping provides a quick fix to the gradient exploding

```
norm = torch.sqrt(
    sum(torch.sum((p.grad ** 2)) for p in params))
if norm > theta:
    for param in params:
        param.grad[:] *= theta / norm
```